In [46]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import plotly.express as px

On récupére le dataframe complétement nettoyé

In [47]:
df = pd.read_csv('csv_files/atp_data_4.csv', sep = ";")
df.drop(['Unnamed: 0'], axis =1, inplace = True)

/Users/guillaumegreau/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning:

Columns (29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44) have mixed types.Specify dtype option on import or set low_memory=False.



On sélectionne seulement les variables qui vont nous permettre d'implémenter toutes les variables souhaitées

In [48]:
df = df[['Year', 'Week_number', 'Year_week', 'Date', 'Location',
       'Tournament', 'Series', 'Court', 'Surface', 'Round', 'Best of', 'Winner', 'Loser']]

On crée un dataframe qui référence tous les matchs de chacun des joueurs entre 2000 et 2021 avec toutes les caractéristiques du match. L'index ici sera le nom de joueurs

In [49]:
df_win = df.drop('Loser', axis = 1)
df_lose = df.drop('Winner', axis =1)

df_win = df_win.assign(Victoire=1, Défaite=0)
df_lose = df_lose.assign(Victoire=0, Défaite=1)

df_win = df_win.rename({'Winner':'Name', }, axis = 1)
df_lose = df_lose.rename({'Loser':'Name'}, axis = 1)

df_players = pd.concat([df_win, df_lose])

df_players.loc[(df_players['Victoire'] == 1), ['Résultats']] = 'V'
df_players.loc[(df_players['Défaite'] == 1), ['Résultats']] = 'D'
df_players = df_players.drop(['Victoire', 'Défaite'], axis = 1)

df_players = df_players.set_index('Name')

Un exemple du dataframe crée. On peut voir apparaître une nouvelle colonne **Résultats** donnant ainsi l'issue du match. Pour un seul match, on aura donc 2 lignes, l'une pour le vainqueur avec la valeur **V** attribuée, l'autre pour le perdant avec la valeur **D**.

In [50]:
df_players.sample(5)

,Year,Week_number,Year_week,Date,Location,Tournament,Series,Court,Surface,Round,Best of,Résultats
Name,,,,,,,,,,,,
Monteiro T.,2017,20,201720,15/05/2017,Rome,Rome Master,M1000,Outdoor,Clay,2R,3,D
Montanes A.,2009,39,200939,22/09/2009,Bucharest,Bucharest Championship,ATP250,Outdoor,Clay,3R,3,V
Draper S.,2002,26,200226,24/06/2002,London,Wimbledon,GS,Outdoor,Grass,1R,5,V
Llodra M.,2008,22,200822,28/05/2008,Paris,Roland Garros,GS,Outdoor,Clay,2R,5,V
Hipfl M.,2000,21,200021,22/05/2000,St. Polten,St. Polten Championship,ATP250,Outdoor,Clay,Q,3,D


In [51]:
df_players.columns

Index(['Year', 'Week_number', 'Year_week', 'Date', 'Location', 'Tournament',
       'Series', 'Court', 'Surface', 'Round', 'Best of', 'Résultats'],
      dtype='object')

Pour faire écho à l'exemple représenté dans le rapport. On voit bien que pour le même match nous avons 2 lignes avec le vainqueur et le perdant

In [52]:
df_players[ (df_players['Tournament'] == 'US Open') & 
            (df_players['Year'] == 2011) &
            (df_players['Round'] == 'SF')].loc[['Federer R.', 'Djokovic N.'],:]

,Year,Week_number,Year_week,Date,Location,Tournament,Series,Court,Surface,Round,Best of,Résultats
Name,,,,,,,,,,,,
Federer R.,2011,36,201136,10/09/2011,New York,US Open,GS,Outdoor,Hard,SF,5,D
Djokovic N.,2011,36,201136,10/09/2011,New York,US Open,GS,Outdoor,Hard,SF,5,V


Le nouveau dataset n'est plus ordonné temporellement donc on le réordonne à nouveau

In [53]:
#transforme la date en type datetime
df_players['Date'] = pd.to_datetime(df_players['Date'], format = "%d/%m/%Y")

#permet de réordonner le dataframe
def create_id_round(tour):
    dictionnaire = {'RR':1, '1R':2, '2R':3, '3R':4, '8th':5, 'Q':6, 'SF':7, 'F':8}
    return dictionnaire[tour]

df_players['id_round'] = df_players['Round'].apply(create_id_round)

df_players = df_players.sort_values(by = ['Date', 'Tournament', 'id_round'])
df_players = df_players.drop('id_round', axis=1)

In [54]:
df_players['Date']

Name
Dosedel S.    2000-01-03
Enqvist T.    2000-01-03
Escude N.     2000-01-03
Federer R.    2000-01-03
Fromberg R.   2000-01-03
                 ...    
Zverev A.     2021-11-20
Ruud C.       2021-11-20
Djokovic N.   2021-11-20
Zverev A.     2021-11-21
Medvedev D.   2021-11-21
Name: Date, Length: 117328, dtype: datetime64[ns]

In [55]:
df_players.index.value_counts().index

Index(['Federer R.', 'Nadal R.', 'Djokovic N.', 'Ferrer D.', 'Verdasco F.',
       'Lopez F.', 'Berdych T.', 'Gasquet R.', 'Youzhny M.', 'Murray A.',
       ...
       'Xu J.C.', 'Ignatik U.', 'Estrella V.', 'Piccari F.', 'O'Neal J.',
       'Brizzi A.', 'Kim K', 'Middelkoop M.', 'Lipsky S.', 'Durasovic V.'],
      dtype='object', length=1532)

On crée le système d'attribution des points ATP et on attribue à chaque joueur subissant une défaite, le
nombre de points qu'il obtient en fonction de son parcours dans un tournoi

In [56]:
atp_points = {'ATP250' : [0 ,  0, 10,  20,  45,  90,  150,   250, np.nan, np.nan],
              'ATP500' : [0 ,  0, 20,  45,  90, 180,  300,   500, np.nan, np.nan],
              'M1000' :  [10, 25, 45,  90, 180, 360,  600,  1000, np.nan, np.nan],
              'GS' :     [10, 45, 90, 180, 360, 720, 1200,  2000, np.nan, np.nan],
              'ATP Final': [np.nan, np.nan, np.nan, np.nan, np.nan, 0, 400, 500, 0, 200]}
atp_points = pd.DataFrame(data = atp_points, index = ['D1R', 'D2R', 'D3R', 'D8th', 'DQ', 'DSF', 'DF', 'VF', 'DRR', 'VRR'])

#essaie de rajouter le nombre de points gagnés pour chacun des joueurs à chacun de leurs matchs
df_players['ATP_Points'] = df_players['Résultats'] + '-' + df_players['Series'] + '-' + df_players['Round']

#essaie de rajouter le nombre de points gagnés pour chacun des joueurs à chacun de leurs matchs
df_players['ATP_Points'] = df_players['Résultats'] + '-' + df_players['Series'] + '-' + df_players['Round']

def attribution_points(points) :

    (resultat, serie, tour) = tuple(points.split('-'))

    if resultat == 'D'  :
        if resultat != 'RR' :
            return atp_points.loc[resultat+tour, serie]
        else :
            return 0
    elif (resultat == 'V') :
        if (tour == 'RR') | (tour == 'F') :
            return atp_points.loc[resultat+tour, serie]
        else :
            return 0
        
        print()

#permet d'attributer les points ATP à chaque matchs joués
df_players['ATP_Points'] = df_players['ATP_Points'].apply(attribution_points)

On renomme la colonne **Best of** par **Format**. C'est la colonne qui donne l'information sur le nombre de sets
joués durant le match : 3 ou 5 sets.

In [57]:
df_players.rename(columns={'Best of':'Format'}, inplace = True)

# Étude statistique des caractéristiques des joueurs

Noms des joueurs pour lesquels nous allons étudier leurs statistiques

In [58]:
#on implemente le nom des 2 joueurs dont on souhaite étudier les caractéristiques
player_1 = 'Seppi A.'
player_2 = 'Federer R.'
player_3 = 'Ferrer D.'
player_4 = 'Simon G.'
player_5 = 'Cilic M.'

color = {player_1 : "darkslateblue", player_2 : "indianred", player_3 : "darkseagreen", player_4 : "khaki", 
         player_5 : "cornflowerblue"}

Fonction qui retourne une crosstab dans laquelle se trouve les résultats obtenus par les joueurs en fonction de la variable que l'on souhaite étudier

In [59]:
#fonction qui permet de créer une crosstab entre la variable que l'on souhaite étudier et le pourcentage
#de victoires/défaites de ces 2 joueurs pour cette variable en particulier

def create_crosstab(x, y, player_1, player_2, player_3, player_4, player_5, df_players) :
    
    stats_player_1 = df_players.loc[player_1,:].rename(columns = {x:'Player'})
    stats_player_2 = df_players.loc[player_2,:].rename(columns = {x:'Player'})
    stats_player_3 = df_players.loc[player_3,:].rename(columns = {x:'Player'})
    stats_player_4 = df_players.loc[player_4,:].rename(columns = {x:'Player'})
    stats_player_5 = df_players.loc[player_5,:].rename(columns = {x:'Player'})
    
    crosstab_1 = pd.crosstab(stats_player_1[y], stats_player_1['Player'], normalize = 0)
    crosstab_2 = pd.crosstab(stats_player_2[y], stats_player_2['Player'], normalize = 0)
    crosstab_3 = pd.crosstab(stats_player_3[y], stats_player_3['Player'], normalize = 0)
    crosstab_4 = pd.crosstab(stats_player_4[y], stats_player_4['Player'], normalize = 0)
    crosstab_5 = pd.crosstab(stats_player_5[y], stats_player_5['Player'], normalize = 0)
    
    crosstab_1 = crosstab_1.drop('D', axis = 1).rename(columns = {'V':player_1})
    crosstab_2 = crosstab_2.drop('D', axis = 1).rename(columns = {'V':player_2})
    crosstab_3 = crosstab_3.drop('D', axis = 1).rename(columns = {'V':player_3})
    crosstab_4 = crosstab_4.drop('D', axis = 1).rename(columns = {'V':player_4})
    crosstab_5 = crosstab_5.drop('D', axis = 1).rename(columns = {'V':player_5})
    
    crosstab_players = crosstab_1.merge(right=crosstab_2, on = y, how = 'outer')
    crosstab_players = crosstab_players.merge(right=crosstab_3, on = y, how = 'outer')
    crosstab_players = crosstab_players.merge(right=crosstab_4, on = y, how = 'outer')
    crosstab_players = crosstab_players.merge(right=crosstab_5, on = y, how = 'outer')
    
    crosstab_players = crosstab_players.reset_index().rename({y:' '}, axis = 1).set_index(' ')
    
    return crosstab_players

def create_dataframe(bar, crosstab) :

    liste_var = list(crosstab.index)
    liste_df = []

    for var in liste_var :
        liste_var_copy = liste_var.copy()
        liste_var_copy.remove(var)
        df_bar = crosstab.transpose().reset_index().drop(liste_var_copy, axis = 1)
        df_bar = df_bar.rename(columns={var:'Pourcentage'})
        df_bar[bar] = [var]*5
        liste_df.append(df_bar)

    return pd.concat(liste_df, axis=0)

Fonction qui permet d'afficher tous les graphes qui vont suivre

In [60]:
def display_plot_bar(crosstab, nb_plot_bar, figsize, width, color, rot, name_img) :
    
    for i in range(nb_plot_bar) :
        
        modal = crosstab.index[i]
        
        crosstab.loc[[modal],:].sort_values(by=modal, axis = 1).plot.bar(
                 ax= axes[i], color = color, rot = rot, figsize = figsize, width = width,
                 title = '')
        
        if i == 0 :
            axes[i].set_ylim([0, 1])
            axes[i].set_title(modal)
            axes[i].set_ylabel('Pourcentages')
            axes[i].set_title('')
    
        else : 
            axes[i].set_ylim([0, 1])
            axes[i].set_title(modal)
            axes[i].yaxis.set_visible(False)
            axes[i].legend().set_visible(False)
            axes[i].set_title('')

    plt.savefig(name_img)
    plt.show()
    

On utilise les 2 fonctions précedentes pour tracer les statistiques des joueurs. Chaque cellule correspond à une variable étudiée

Variable **court**.

In [61]:
vertical_concat = create_dataframe('Court', crosstab_court)

fig = px.bar(vertical_concat, x='Court', y='Pourcentage', color = 'Player', barmode='group')
fig.update_layout(title_font_size=24)
fig.update_xaxes(showgrid=True)
fig.show()

Variable **surface**.

In [62]:
vertical_concat = create_dataframe('Surface', crosstab_surface)

fig = px.bar(vertical_concat, x='Surface', y='Pourcentage', color = 'Player', barmode='group')
fig.update_layout(title_font_size=24)
fig.update_xaxes(showgrid=True)
fig.show()

Variable **round** avec la condition que **serie** = GC.

In [63]:
vertical_concat = create_dataframe('Round', crosstab_round_GS)

fig = px.bar(vertical_concat, x='Round', y='Pourcentage', color = 'Player', barmode='group')
fig.update_layout(title_font_size=24)
fig.update_xaxes(showgrid=True)
fig.show()

Variable **Format**.

In [64]:
vertical_concat = create_dataframe('Format', crosstab_format)

fig = px.bar(vertical_concat, x='Format', y='Pourcentage', color = 'Player', barmode='group')
fig.update_layout(title_font_size=24)
fig.update_xaxes(showgrid=True)
fig.show()

Variable **tournament** avec la condition **serie** = M1000.

In [65]:
vertical_concat = create_dataframe('Tournament', crosstab_m1000)

fig = px.bar(vertical_concat, x='Tournament', y='Pourcentage', color = 'Player', barmode='group')
fig.update_layout(title_font_size=24)
fig.update_xaxes(showgrid=True)
fig.show()

Face à face entre différents joueurs.

In [66]:
player1 = 'Djokovic N.'
player2 = 'Nadal R.'

df_h2h = df[ ((df['Winner']==player1) & (df['Loser']==player2)) |
             ((df['Winner']==player2) & (df['Loser']==player1)) ]

df_h2h_win = df_h2h.drop('Loser', axis = 1)
df_h2h_lose = df_h2h.drop('Winner', axis =1)

df_h2h_win = df_h2h_win.assign(Victoire=1, Défaite=0)
df_h2h_lose = df_h2h_lose.assign(Victoire=0, Défaite=1)

df_h2h_win = df_h2h_win.rename({'Winner':'Player', }, axis = 1)
df_h2h_lose = df_h2h_lose.rename({'Loser':'Player'}, axis = 1)

df_players_h2h = pd.concat([df_h2h_win, df_h2h_lose])
df_players_h2h = df_players_h2h.groupby(['Surface', 'Player']).sum().reset_index()
df_players_h2h.drop('Défaite', axis=1, inplace=True)

fig = px.bar(df_players_h2h, x="Player", y="Victoire", color="Surface", color_discrete_map={'Clay':'darkorange',
                                 'Grass':'forestgreen',
                                 'Hard':'cornflowerblue','Carpet':'plum'})
fig.show()


# Création de toutes les variables

Fonctions utilisées pour créer les variables

In [67]:
def select_df_date(data_frame, colonne, date, cut_week) :
    
    cut_date = date - datetime.timedelta(weeks=cut_week)
    return data_frame[ (data_frame['Date'] >= cut_date) & (data_frame['Date'] <  date) ]

def win_lose_rang(df_player_date_inf, colonne):

    if len(df_player_date_inf) == 0 :
        return 0, 0

    else :
        df_player_cut_reverse = df_player_date_inf.loc[::-1]
        df_player_cut_reverse = df_player_cut_reverse.reset_index()

        last_match = df_player_cut_reverse.iloc[0,:]['Résultats']
    
        if len(df_player_cut_reverse) == 1 :
            if last_match == 'V' : 
                return 1, 0
            elif last_match == 'D' :
                return 0, 1
        
        else : 
            
            size_df = len(df_player_cut_reverse)
            i = 0
            count = 0
    
            if last_match == 'V' :
                while (last_match == 'V') & (i < size_df) :
                    
                    i += 1
                    count += 1
                    
                    if i < size_df :
                        last_match = df_player_cut_reverse.iloc[i,:]['Résultats']
                    
                return count, 0

            elif last_match == 'D' :
                
                while (last_match == 'D') & (i < size_df) :
                    
                    i += 1
                    count += 1
                    
                    if i < size_df :
                        last_match = df_player_cut_reverse.iloc[i,:]['Résultats']

                return 0, count

Code principal pour créer les variables. Ici le code ne contient que les 2 dernières variables crées. Toutes les précédentes ont également été crée à partir de ce code mais n'apparaissent plus car le code était trop long. Les variables n'ont pas été crée toutes en même temps mais au fur et à mesure et le temps d'éxécution était d'environ une heure pour tourner et ce pour seulement 3-4 variables.
Le temps d'exécution est long car il boucle sur chacune des lignes du dataset et pour chacune de ces lignes, il vient faire des opérations sur toutes les lignes qui se trouvent au-dessus car chaque information recueillie sur un joueur dépend de ses matchs passés. 

**Évitez de compiler cette cellule et passez à la cellule d'après directement**

In [33]:
import time

# starting time
start = time.time()

values = []

year = 2000

for player, date, tournament in zip(df_players.index, df_players['Date'], df_players['Tournament']):
    
    if date.year != year :
        print(date.year)
        year = date.year
        
    df_player = df_players[df_players.index == player]
    
    #df_player_date_4 = select_df_date(df_player, 'Date', date, 17)
    #df_player_date_8 = select_df_date(df_player, 'Date', date, 35)
    #df_player_date_12 = select_df_date(df_player, 'Date', date, 52)
    df_player_date_260 = select_df_date(df_player, 'Date', date, 260)
    
    v_RR_ATP_final_5y = df_player_date_260.query('Résultats == "V" & Series == "ATP Final" & Round == "RR"').Résultats.count()
    d_RR_ATP_final_5y = df_player_date_260.query('Résultats == "D" & Series == "ATP Final" & Round == "RR"').Résultats.count()
    
    v_SF_ATP_final_5y = df_player_date_260.query('Résultats == "V" & Series == "ATP Final" & Round == "SF"').Résultats.count()
    d_SF_ATP_final_5y = df_player_date_260.query('Résultats == "D" & Series == "ATP Final" & Round == "SF"').Résultats.count()
    
    v_F_ATP_final_5y = df_player_date_260.query('Résultats == "V" & Series == "ATP Final" & Round == "F"').Résultats.count()
    d_F_ATP_final_5y = df_player_date_260.query('Résultats == "D" & Series == "ATP Final" & Round == "F"').Résultats.count()
    
    v_tournament_5y = df_player_date_260 [ df_player_date_260['Tournament'] == tournament ].query('Résultats == "V"').Résultats.count()
    d_tournament_5y = df_player_date_260 [ df_player_date_260['Tournament'] == tournament ].query('Résultats == "D"').Résultats.count()
    
    values.append(( 
                   v_RR_ATP_final_5y - d_RR_ATP_final_5y,
                   v_SF_ATP_final_5y - d_SF_ATP_final_5y,
                   v_F_ATP_final_5y - d_F_ATP_final_5y,
                   v_tournament_5y - d_tournament_5y,
        
                   v_RR_ATP_final_5y/(v_RR_ATP_final_5y + d_RR_ATP_final_5y),
                   v_SF_ATP_final_5y/(v_SF_ATP_final_5y + d_SF_ATP_final_5y),
                   v_F_ATP_final_5y/(v_F_ATP_final_5y + d_F_ATP_final_5y),
                   v_tournament_5y/(v_tournament_5y + d_tournament_5y)
        
                  ))

data_values = pd.DataFrame(values, columns=('V-D_RR_ATP_final_5y', 'V-D_SF_ATP_final_5y',
                                            'V-D_F_ATP_final_5y', 'V-D_tournament_5y',
                                            
                                            'ratioV_RR_ATP_final_5y', 'ratioV_SF_ATP_final_5y',
                                            'ratioV_F_ATP_final_5y', 'ratioV_tournament_5y' 
                                
                                           ))

end = time.time()

# total time taken
print(f"Runtime of the program is {end - start}")

/var/folders/vt/4q_hb0nn2dbb6_vm__4gc9tm0000gn/T/ipykernel_87706/1236901542.py:41: RuntimeWarning: invalid value encountered in long_scalars
  v_RR_ATP_final_5y/(v_RR_ATP_final_5y + d_RR_ATP_final_5y),
/var/folders/vt/4q_hb0nn2dbb6_vm__4gc9tm0000gn/T/ipykernel_87706/1236901542.py:42: RuntimeWarning: invalid value encountered in long_scalars
  v_SF_ATP_final_5y/(v_SF_ATP_final_5y + d_SF_ATP_final_5y),
/var/folders/vt/4q_hb0nn2dbb6_vm__4gc9tm0000gn/T/ipykernel_87706/1236901542.py:43: RuntimeWarning: invalid value encountered in long_scalars
  v_F_ATP_final_5y/(v_F_ATP_final_5y + d_F_ATP_final_5y),
/var/folders/vt/4q_hb0nn2dbb6_vm__4gc9tm0000gn/T/ipykernel_87706/1236901542.py:44: RuntimeWarning: invalid value encountered in long_scalars
  v_tournament_5y/(v_tournament_5y + d_tournament_5y)


KeyboardInterrupt: 

La dernière cellule en dessous permet de récupérer le dataset stocké qui contient toutes les variables crées au fur et à mesure

In [43]:
df_players = pd.read_csv('csv_files/atp_data_all_players_var.csv', sep=';')
df_players.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [44]:
df_players

,Name,Year,Week_number,Year_week,Date,Location,Tournament,Series,Court,Surface,...,V-D_3y_f5,ratioV_3y,ratioV_3y_hard,ratioV_3y_clay,ratioV_3y_grass,ratioV_3y_carpet,ratioV_3y_indoor,ratioV_3y_outdoor,ratioV_3y_f3,ratioV_3y_f5
0,Dosedel S.,2000,1,200001,2000-01-03,Adelaide,Adelaide Championship,ATP250,Outdoor,Hard,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Enqvist T.,2000,1,200001,2000-01-03,Adelaide,Adelaide Championship,ATP250,Outdoor,Hard,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Escude N.,2000,1,200001,2000-01-03,Adelaide,Adelaide Championship,ATP250,Outdoor,Hard,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Federer R.,2000,1,200001,2000-01-03,Adelaide,Adelaide Championship,ATP250,Outdoor,Hard,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fromberg R.,2000,1,200001,2000-01-03,Adelaide,Adelaide Championship,ATP250,Outdoor,Hard,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117323,Zverev A.,2021,46,202146,2021-11-20,Turin,Turin ATP Final,ATP Final,Indoor,Hard,...,30,0.719512,0.738318,0.717391,0.545455,NaN,0.694444,0.726562,0.687500,0.788462
117324,Ruud C.,2021,46,202146,2021-11-20,Turin,Turin ATP Final,ATP Final,Indoor,Hard,...,2,0.666667,0.528302,0.775000,0.400000,NaN,0.583333,0.684211,0.689655,0.545455
117325,Djokovic N.,2021,46,202146,2021-11-20,Turin,Turin ATP Final,ATP Final,Indoor,Hard,...,60,0.871795,0.855556,0.865385,1.000000,NaN,0.782609,0.887218,0.825581,0.928571
117326,Zverev A.,2021,46,202146,2021-11-21,Turin,Turin ATP Final,ATP Final,Indoor,Hard,...,30,0.721212,0.740741,0.717391,0.545455,NaN,0.702703,0.726562,0.690265,0.788462


Aperçu de toutes les variables crées avec ce code 

In [45]:
df_players.columns

Index(['Name', 'Year', 'Week_number', 'Year_week', 'Date', 'Location',
       'Tournament', 'Series', 'Court', 'Surface', 'Round', 'Format',
       'Résultats', 'V-D_3y', 'V-D_3y_hard', 'V-D_3y_clay', 'V-D_3y_grass',
       'V-D_3y_carpet', 'V-D_3y_indoor', 'V-D_3y_outdoor', 'V-D_m1000_1R_5y',
       'V-D_m1000_2R_5y', 'V-D_m1000_3R_5y', 'V-D_m1000_8th_5y',
       'V-D_m1000_Q_5y', 'V-D_m1000_SF_5y', 'V-D_m1000_F_5y', 'V-D_gs_1R_5y',
       'V-D_gs_2R_5y', 'V-D_gs_3R_5y', 'V-D_gs_8th_5y', 'V-D_gs_Q_5y',
       'V-D_gs_SF_5y', 'V-D_gs_F_5y', 'ratioV_m1000_1R_5y',
       'ratioV_m1000_2R_5y', 'ratioV_m1000_3R_5y', 'ratioV_m1000_8th_5y',
       'ratioV_m1000_Q_5y', 'ratioV_m1000_SF_5y', 'ratioV_m1000_F_5y',
       'ratioV_gs_1R_5y', 'ratioV_gs_2R_5y', 'ratioV_gs_3R_5y',
       'ratioV_gs_8th_5y', 'ratioV_gs_Q_5y', 'ratioV_gs_SF_5y',
       'ratioV_gs_F_5y', 'last_match_played', 'matchs_played_3w', 'V-D_3w',
       'ratioV_3w', 'v-d_rang', 'V-D_RR_ATP_final_5y', 'V-D_SF_ATP_final_5y',
    